In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:

import re

import torch
import numpy as np
import tensorflow as tf

In [3]:
m0 = torch.load('./model-v1/80000/mp_rank_00_model_states.pt', map_location='cpu')
m1 = torch.load('./model-v1/80000/mp_rank_01_model_states.pt', map_location='cpu')

In [4]:
def get_weight(model, name):
    for n, w in model['module'].items():
        if name == n:
            return w

In [5]:
for n, w in m0['module'].items():
    if '.layers.' in n:
        if '.layers.0.' in n:
            print(n, w.shape)
    else:
        print(n, w.shape)

word_embeddings.weight torch.Size([15000, 2560])
position_embeddings.weight torch.Size([1024, 2560])
transformer.layers.0.input_layernorm.weight torch.Size([2560])
transformer.layers.0.input_layernorm.bias torch.Size([2560])
transformer.layers.0.attention.query_key_value.weight torch.Size([3840, 2560])
transformer.layers.0.attention.query_key_value.bias torch.Size([3840])
transformer.layers.0.attention.dense.weight torch.Size([2560, 1280])
transformer.layers.0.attention.dense.bias torch.Size([2560])
transformer.layers.0.post_attention_layernorm.weight torch.Size([2560])
transformer.layers.0.post_attention_layernorm.bias torch.Size([2560])
transformer.layers.0.mlp.dense_h_to_4h.weight torch.Size([5120, 2560])
transformer.layers.0.mlp.dense_h_to_4h.bias torch.Size([5120])
transformer.layers.0.mlp.dense_4h_to_h.weight torch.Size([2560, 5120])
transformer.layers.0.mlp.dense_4h_to_h.bias torch.Size([2560])
transformer.final_layernorm.weight torch.Size([2560])
transformer.final_layernorm.bia

In [6]:
from transformers import GPT2Config, TFGPT2LMHeadModel

gpt2_config = GPT2Config(
    vocab_size=30000,
    n_positions=1024,
    n_ctx=1024,
    n_embd=2560,
    n_layer=32,
    n_head=32,
    pad_token_id=0,
)
gpt2_model = TFGPT2LMHeadModel(gpt2_config)
loss = gpt2_model.compute_loss
optimizer = tf.keras.optimizers.Adam()

metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# metric = Mymetrice('accuracy')

gpt2_model.compile(
    optimizer=optimizer,
    loss=[loss, *[None] * gpt2_model.config.n_layer],
    metrics=[metric]
)
input = tf.constant([[1, 2]])
out = gpt2_model(input)[0]
print(out.shape)

(1, 2, 30000)


In [7]:
for w in gpt2_model.weights:
    if 'h_._' in w.name:
        if 'h_._0' in w.name or 'h_._31' in w.name:
            print(w.name, w.shape)
    else:
        print(w.name, w.shape)

tfgp_t2lm_head_model/transformer/wte/weight:0 (30000, 2560)
tfgp_t2lm_head_model/transformer/wpe/embeddings:0 (1024, 2560)
tfgp_t2lm_head_model/transformer/h_._0/ln_1/gamma:0 (2560,)
tfgp_t2lm_head_model/transformer/h_._0/ln_1/beta:0 (2560,)
tfgp_t2lm_head_model/transformer/h_._0/attn/c_attn/weight:0 (2560, 7680)
tfgp_t2lm_head_model/transformer/h_._0/attn/c_attn/bias:0 (1, 7680)
tfgp_t2lm_head_model/transformer/h_._0/attn/c_proj/weight:0 (2560, 2560)
tfgp_t2lm_head_model/transformer/h_._0/attn/c_proj/bias:0 (1, 2560)
tfgp_t2lm_head_model/transformer/h_._0/ln_2/gamma:0 (2560,)
tfgp_t2lm_head_model/transformer/h_._0/ln_2/beta:0 (2560,)
tfgp_t2lm_head_model/transformer/h_._0/mlp/c_fc/weight:0 (2560, 10240)
tfgp_t2lm_head_model/transformer/h_._0/mlp/c_fc/bias:0 (1, 10240)
tfgp_t2lm_head_model/transformer/h_._0/mlp/c_proj/weight:0 (10240, 2560)
tfgp_t2lm_head_model/transformer/h_._0/mlp/c_proj/bias:0 (1, 2560)
tfgp_t2lm_head_model/transformer/h_._31/ln_1/gamma:0 (2560,)
tfgp_t2lm_head_mode

In [ ]:
new_weights = []
for w in gpt2_model.weights:
    num_layer = 0
    if 'h_._' in w.name:
        num_layer = re.findall(r'transformer/h_._(\d+)', w.name)[0]

    if 'transformer/wte/weight:0' in w.name:
        w0 = get_weight(m0, f'word_embeddings.weight')
        w1 = get_weight(m1, f'word_embeddings.weight')
        w = np.concatenate([w0.numpy(), w1.numpy()])
        assert w.shape == (30000, 2560)
        new_weights.append(w)
        print(f'setting transformer/wte/weight')
    elif 'transformer/wpe/embeddings:0' in w.name:
        w0 = get_weight(m0, f'position_embeddings.weight')
        ｗ = w0.numpy()
        assert w.shape == (1024, 2560)
        new_weights.append(w)
        print(f'setting transformer/wpe/embeddings')
    elif 'ln_1/gamma:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.input_layernorm.weight')
        w = w0.numpy()
        assert w.shape == (2560, )
        new_weights.append(w)
        print(f'setting layer {num_layer} ln_1/gamma')
    elif 'ln_1/beta:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.input_layernorm.bias')
        w = w0.numpy()
        assert w.shape == (2560, )
        new_weights.append(w)
        print(f'setting layer {num_layer} ln_1/beta')
    elif 'attn/c_attn/weight:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.attention.query_key_value.weight')
        w1 = get_weight(m1, f'transformer.layers.{num_layer}.attention.query_key_value.weight')
        w0 = w0.numpy()
        w1 = w1.numpy()
        q0 = w0[:1280, :]
        q1 = w1[:1280, :]
        k0 = w0[1280:1280 * 2, :]
        k1 = w1[1280:1280 * 2, :]
        v0 = w0[1280 * 2:, :]
        v1 = w1[1280 * 2:, :]
        w = np.concatenate([q0, q1, k0, k1, v0, v1])
        assert w.shape == (7680, 2560)
        w = np.transpose(w)
        new_weights.append(w)
        print(f'setting layer {num_layer} c_attn/weight')
    elif 'attn/c_attn/bias:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.attention.query_key_value.bias')
        w1 = get_weight(m1, f'transformer.layers.{num_layer}.attention.query_key_value.bias')
        w = np.concatenate([w0.numpy(), w1.numpy()])
        w = np.transpose(w)
        w = w.reshape(1, 7680)
        new_weights.append(w)
        print(f'setting layer {num_layer} c_attn/bias')
    elif 'attn/c_proj/weight:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.attention.dense.weight')
        w1 = get_weight(m1, f'transformer.layers.{num_layer}.attention.dense.weight')
        w = np.concatenate([w0.numpy(), w1.numpy()], axis=-1)
        assert w.shape == (2560, 2560)
        w = np.transpose(w)
        new_weights.append(w)
        print(f'setting layer {num_layer} c_proj/weight')
    elif 'attn/c_proj/bias:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.attention.dense.bias')
        w = w0.numpy()
        assert w.shape == (2560, )
        w = w.reshape(1, 2560)
        new_weights.append(w)
        print(f'setting layer {num_layer} c_proj/bias')
    elif 'ln_2/gamma:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.post_attention_layernorm.weight')
        w = w0.numpy()
        assert w.shape == (2560, )
        new_weights.append(w)
        print(f'setting layer {num_layer} ln_2/gamma')
    elif 'ln_2/beta:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.post_attention_layernorm.bias')
        w = w0.numpy()
        assert w.shape == (2560, )
        new_weights.append(w)
        print(f'setting layer {num_layer} ln_2/beta')
    elif 'mlp/c_fc/weight:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.mlp.dense_h_to_4h.weight')
        w1 = get_weight(m1, f'transformer.layers.{num_layer}.mlp.dense_h_to_4h.weight')
        w = np.concatenate([w0.numpy(), w1.numpy()])
        assert w.shape == (10240, 2560)
        w = np.transpose(w)
        new_weights.append(w)
        print(f'setting layer {num_layer} mlp/c_fc/weight')
    elif 'mlp/c_fc/bias:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.mlp.dense_h_to_4h.bias')
        w1 = get_weight(m1, f'transformer.layers.{num_layer}.mlp.dense_h_to_4h.bias')
        w = np.concatenate([w0.numpy(), w1.numpy()])
        assert w.shape == (10240, )
        w = w.reshape(1, 10240)
        new_weights.append(w)
        print(f'setting layer {num_layer} mlp/c_fc/bias')
    elif 'mlp/c_proj/weight:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.mlp.dense_4h_to_h.weight')
        w1 = get_weight(m1, f'transformer.layers.{num_layer}.mlp.dense_4h_to_h.weight')
        w = np.concatenate([w0.numpy(), w1.numpy()], axis=-1)
        assert w.shape == (2560, 10240)
        w = np.transpose(w)
        new_weights.append(w)
        print(f'setting layer {num_layer} mlp/c_proj/weight')
    elif 'mlp/c_proj/bias:0' in w.name:
        w0 = get_weight(m0, f'transformer.layers.{num_layer}.mlp.dense_4h_to_h.bias')
        w = w0.numpy()
        assert w.shape == (2560, )
        w = w.reshape(1, 2560)
        new_weights.append(w)
        print(f'setting layer {num_layer} mlp/c_proj/bias')
    elif 'transformer/ln_f/gamma:0' in w.name:
        w0 = get_weight(m0, f'transformer.final_layernorm.weight')
        ｗ = w0.numpy()
        assert w.shape == (2560, )
        new_weights.append(w)
        print(f'setting transformer/ln_f/gamma')
    elif 'transformer/ln_f/beta:0' in w.name:
        w0 = get_weight(m0, f'transformer.final_layernorm.bias')
        ｗ = w0.numpy()
        assert w.shape == (2560, )
        new_weights.append(w)
        print(f'setting transformer/ln_f/beta')



In [10]:
gpt2_model.set_weights(new_weights)

In [38]:
from gpt_tokenizer import GPT2Tokenizer

tokenizer = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model'
)

In [60]:
ids = tokenizer.encode('今天天气不错')

for i in range(10):
    output = model(tf.constant([ids]))
    nid = np.argmax(output[0][0, -1])
    ids += [nid]
    print(ids)
    print(i, tokenizer.decode(ids))

[837, 259, 497, 11366, 8]
0 今天天气不错
[837, 259, 497, 11366, 8, 9]
1 今天天气不错,
[837, 259, 497, 11366, 8, 9, 2187]
2 今天天气不错,心情
[837, 259, 497, 11366, 8, 9, 2187, 26]
3 今天天气不错,心情也
[837, 259, 497, 11366, 8, 9, 2187, 26, 788]
4 今天天气不错,心情也不错
[837, 259, 497, 11366, 8, 9, 2187, 26, 788, 8]
5 今天天气不错,心情也不错
[837, 259, 497, 11366, 8, 9, 2187, 26, 788, 8, 9]
6 今天天气不错,心情也不错,
[837, 259, 497, 11366, 8, 9, 2187, 26, 788, 8, 9, 29]
7 今天天气不错,心情也不错,就
[837, 259, 497, 11366, 8, 9, 2187, 26, 788, 8, 9, 29, 84]
8 今天天气不错,心情也不错,就想
[837, 259, 497, 11366, 8, 9, 2187, 26, 788, 8, 9, 29, 84, 197]
9 今天天气不错,心情也不错,就想着


In [38]:
gpt2_model.save_pretrained('/data2/CPM-TF/models')

In [ ]:
from transformers import XLNetTokenizer, TFGPT2LMHeadModel, TextGenerationPipeline


tokenizer = XLNetTokenizer('CPM-Generate/bpe_3w_new/chinese_vocab.model', add_special_token=False)

model = TFGPT2LMHeadModel.from_pretrained("/data2/CPM-TF/models")

text_generater = TextGenerationPipeline(model, tokenizer)

In [9]:
display(text_generater('今天天气不错', max_length=20))
display(text_generater('今天天气不错', max_length=20, do_sample=True, top_k=10, top_p=0.95))
display(text_generater('修真者到了元婴期之后经天地元气重筑的肉身', max_length=60))

[{'generated_text': '今天天气不错 , 心情 也 不错 , 就 想着 出去 散散心 ,'}]

[{'generated_text': '今天天气不错 , 心情舒畅 , 于是 就 想着 吃点 啥 ,'}]

[{'generated_text': '修真者到了元婴期之后经天地元气重筑的肉身 , 因为 元婴 的 境界 , 是 “ 物我两忘 ” , 物我两忘 , 就是 物我两忘 , 物我'}]